#### Purpose: The resulting metadata will contribute to feature engineering for an event-level model

The data were originally pasted as one value per row in groups of 6 rows that need pivoting.

This notebook saves the data out as a structured table

In [1]:
import pandas as pd
from datetime import timedelta
import pickle

In [2]:
onecol=[]
with open('MetaTable/flat.txt') as f:
    Lines = f.readlines()

    for line in Lines:
        onecol.append(line.replace("\n",""))

f.close()

indx = [int(n / 6) for n in range(len(onecol))]

df = pd.DataFrame({'group':indx,'raw':onecol}).set_index('group')

col_names = df[df.index==0]['raw'].values.tolist()
col_names = col_names*147

df = df[df.index>0]

df['col_names'] = col_names

df.head()

,raw,col_names
group,,
1,01/09/2018,flood
1,17 213 562,exposed (mn)
1,1 000,displaced (k)
1,20,killed
1,6,duration(days)


In [3]:

meta_df = df.reset_index().pivot(index='group', columns='col_names', values='raw').reset_index().drop(columns=['group'])
meta_df.rename(index={'col_names': 'gg'}, inplace=True)
meta_df.rename(columns={'displaced (k)': 'displaced_k',
                        'duration(days)': 'duration_days',
                        'exposed (mn)': 'exposed_mn'
                        }, inplace=True)


meta_df.index.names = ['index']
meta_df.columns.names = ['']

for col in ['displaced_k','duration_days','exposed_mn']:
    meta_df[col] = meta_df[col].apply(lambda x: int(x.replace(" ","")))

meta_df['start_date']= pd.to_datetime(meta_df['flood'],format="%d/%m/%Y")

meta_df['end_date']= meta_df[['start_date','duration_days']].apply(lambda x : x.start_date + timedelta(days=x.duration_days), axis=1)# + timedelta(days=1)

meta_df.sort_values('start_date', inplace=True)

meta_df.head()

,cause,displaced_k,duration_days,exposed_mn,flood,killed,start_date,end_date
index,,,,,,,,
146,Heavy rain,24000000,33,7481712,18/09/2000,1 468,2000-09-18,2000-10-21
145,Heavy rain,2000000,33,208434,08/07/2001,100,2001-07-08,2001-08-10
144,Heavy rain,300000,22,1701849,20/08/2001,65,2001-08-20,2001-09-11
143,Heavy rain,30000,17,250633,07/09/2001,146,2001-09-07,2001-09-24
142,Heavy rain,15000000,68,7191176,21/06/2002,503,2002-06-21,2002-08-28


In [4]:
with open('meta_floods_df.pkl', 'wb') as f:  
    pickle.dump(meta_df, f) 

f.close()